In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_excel('/Users/shaikmohammadasrarahammad/Downloads/Crescent subjects/sem 6/EDA/prods.xlsx')

In [3]:
df.head()

,Products produced (in million),Profits gained(in million),MAR,MCAR,MNAR
0,52,15,NaN,NaN,NaN
1,55,4,NaN,4.0,4.0
2,56,11,NaN,11.0,NaN
3,56,11,NaN,11.0,NaN
4,63,6,NaN,6.0,6.0


In [35]:
df_describe = df['Profits gained(in million)'].describe()
df_describe

count    50.000000
mean      9.160000
std       3.908964
min       1.000000
25%       7.000000
50%      10.000000
75%      12.000000
max      15.000000
Name: Profits gained(in million), dtype: float64

In [4]:
df.isnull().sum()

Products produced (in million)    0
Profits gained(in million)        0
MAR                               9
MCAR                              9
MNAR                              9
dtype: int64

# Missing at Random

In [5]:
mar_df = df[['Products produced (in million)','MAR']]
mar_df.head()

,Products produced (in million),MAR
0,52,NaN
1,55,NaN
2,56,NaN
3,56,NaN
4,63,NaN


In [6]:
mar_df.isnull().sum()

Products produced (in million)    0
MAR                               9
dtype: int64

# Removing null values

In [7]:
rm_null_mar = mar_df.dropna()
rm_null_mar.isnull().sum()
rm_null_mar.rename(columns={'MAR': 'dropna'},inplace=True)
rm_null_mar.head()

/var/folders/kh/nby254ps3kv79sh2q9w5ndh80000gn/T/ipykernel_79096/3223884158.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rm_null_mar.rename(columns={'MAR': 'dropna'},inplace=True)


,Products produced (in million),dropna
9,70,8.0
10,73,8.0
11,74,3.0
12,75,12.0
13,79,6.0


In [8]:
mar_na_det = rm_null_mar.describe()

# Replacing with arthematic mean

In [9]:
avg_mar_df = mar_df.fillna(mar_df['MAR'].mean())
avg_mar_df.rename(columns={'MAR': 'Average'},inplace=True)


In [10]:
mar_avg_det = avg_mar_df.describe()

# Regression Imputation

In [11]:
null = mar_df['MAR'].isnull()
null_df = pd.DataFrame(mar_df['Products produced (in million)'][null])

In [12]:
null_df.shape

(9, 1)

In [13]:
ri_df = mar_df.dropna()
ri_df.shape

(41, 2)

In [14]:
X = ri_df[['Products produced (in million)']]
y = ri_df['MAR']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

In [16]:
reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [17]:
y_pred = reg.predict(null_df)

In [18]:
y_pred

array([9.07304317, 9.07517019, 9.07587919, 9.07587919, 9.08084223,
       9.08155124, 9.08155124, 9.08226025, 9.08580527])

In [19]:
null_df['MAR'] = y_pred

In [20]:
null_df 

,Products produced (in million),MAR
0,52,9.073043
1,55,9.075170
2,56,9.075879
3,56,9.075879
4,63,9.080842
5,64,9.081551
6,64,9.081551
7,65,9.082260
8,70,9.085805


In [21]:
ri_df = pd.concat([ri_df,null_df])
ri_df.rename(columns={'MAR': 'RI'},inplace=True)
print(ri_df.isnull().sum())
print()
print(ri_df.shape)

Products produced (in million)    0
RI                                0
dtype: int64

(50, 2)


In [22]:
ri_mar_det = ri_df.describe()

# Stocastic Regression Imputation

In [23]:
sri_df = mar_df.dropna()
sri_df.shape

(41, 2)

In [24]:
null = mar_df['MAR'].isnull()
null_df = pd.DataFrame(mar_df['Products produced (in million)'][null])

In [25]:
y_pred_noise = reg.predict(null_df) + np.random.randn(9,)

In [26]:
y_pred_noise

array([ 9.0071169 ,  8.79332315, 10.73431933,  9.6067876 ,  8.45979114,
       10.02470344, 10.12619114,  8.67856308,  7.78729067])

In [27]:
null_df['MAR'] = y_pred_noise
null_df

,Products produced (in million),MAR
0,52,9.007117
1,55,8.793323
2,56,10.734319
3,56,9.606788
4,63,8.459791
5,64,10.024703
6,64,10.126191
7,65,8.678563
8,70,7.787291


In [28]:
sri_df = pd.concat([sri_df, null_df])
sri_df.rename(columns={'MAR': 'SRI'},inplace=True)
sri_df.shape

(50, 2)

In [29]:
sri_mar_det = sri_df.describe()

# Conclusion Report of MAR

In [38]:
mar_final = pd.concat([mar_na_det, mar_avg_det['Average'] ,ri_mar_det['RI'] ,sri_mar_det['SRI'],df_describe,],axis=1)
mar_final

,Products produced (in million),dropna,Average,RI,SRI,Profits gained(in million)
count,41.000000,41.000000,50.000000,50.000000,50.000000,50.000000
mean,127.512195,9.097561,9.097561,9.094240,9.124362,9.160000
std,37.257296,4.066970,3.674539,3.674547,3.694534,3.908964
min,70.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,101.000000,7.000000,7.000000,7.000000,7.000000,7.000000
50%,120.000000,9.000000,9.097561,9.078361,9.003558,10.000000
75%,166.000000,12.000000,12.000000,12.000000,12.000000,12.000000
max,194.000000,15.000000,15.000000,15.000000,15.000000,15.000000


# Missing Completely at Random